In [1]:
import pandas as pd
import pyterrier as pt #pip install python-terrier
import os
from ast import literal_eval


if not pt.started():
    pt.init()


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


# Indexação

Iremos utilizar a biblioteca PyTerrier (https://pyterrier.readthedocs.io/) para indexação e experimentos. A biblioteca utiliza o padrão de entrada/saída da TREC. Por isso, algumas transformações nos dados são necessárias, como adequar nomes de colunas, tipos de dados, entre outros.

In [2]:
# vamos indexar titulo do produto, categoria e suas tags (esta ultima, como uma string única)
def concat_tags(x):
    s = ""
    for i in x:
        s += i + " "
    return str(s)
    
products = pd.read_csv("products.csv",dtype={'product_id': str, 'category':str},converters={"tags": literal_eval})
products["docno"] = products["product_id"]
products["text"] = products["title"]
products["category"].fillna("", inplace = True)

products["new_tags"] = products["tags"].apply(lambda x: concat_tags(x))

pd_indexer = pt.DFIndexer("./pd_index")
indexref = pd_indexer.index(products["text"], products["title"], products["new_tags"], products["category"], products["docno"])

21:33:10.207 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (17179875030) - further warnings are suppressed
21:33:36.300 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 2 empty documents


Precisamos gerar um conjunto de "topics": pares qid (query id) e query. 

In [3]:
## gerando topics
pairs = pd.read_csv("pairs.csv",dtype={'product_id': str})
# atribuindo ids pra queries
pairs['qid'] = pairs.groupby(['query']).ngroup()
topics = pairs[["qid","query"]]
topics = topics.drop_duplicates("qid")
topics["qid"] = topics["qid"].astype(str)
topics

,qid,query
0,83,Convite Padrinhos Batismo
1,92,Decoracao De Casamento
2,230,Toalha De Lavabo
3,59,Calendario 2023 Editavel
4,96,Ecobag
...,...,...
1521,131,Lembrancinha Barata De Natal
1550,166,Minha Vida É Uma Viagem
1648,193,Presente De Formatura Infantil
3875,174,Musicas Nacionais Romanticas Anos 80


Também é necessário criar qrels: tuplas com qid, docno (identificador único para cada item), e label de relevância. Para isso, é necessário cruzar as tabelas products e pairs

In [4]:
## gerando qrels
# pandas.Dataframe with columns=[‘qid’,’docno’, ‘label’]
joint_table = products.merge(pairs,on="product_id")
joint_table.head()

,product_id,title,tags,creation_date,price,weight,express_delivery,category,minimum_quantity,print_count_product,...,text,new_tags,pair_id,query,search_position,print_count_query,view_count_query,cart_count_query,order_count_query,qid
0,101,Jogo Banheiro de Crochê de 3 Peças,"[#jogobanheiro #croche #tapetes, decoração, na...",2022-09-25 13:43:36,110.0,1.0,1,Técnicas de Artesanato,1,11,...,Jogo Banheiro de Crochê de 3 Peças,#jogobanheiro #croche #tapetes decoração nas c...,1520418423149,Necessarie,281,143,5,0,0,178
1,101,Jogo Banheiro de Crochê de 3 Peças,"[#jogobanheiro #croche #tapetes, decoração, na...",2022-09-25 13:43:36,110.0,1.0,1,Técnicas de Artesanato,1,11,...,Jogo Banheiro de Crochê de 3 Peças,#jogobanheiro #croche #tapetes decoração nas c...,55,Necessaire,281,104,3,0,0,176
2,106,Guardanapos de Tecido - 100 unidades,"[guardanapos de tecido, guradanapo, festa, eve...",2014-12-26 18:47:48,269.5,0.0,0,Casa,1,62,...,Guardanapos de Tecido - 100 unidades,guardanapos de tecido guradanapo festa evento ...,278,Bebê,86,346,7,1,1,21
3,47,Toalha Papai Noel,"[natal, toalha de natal, toalha de mesa, papai...",2013-11-06 20:43:27,291.1,0.0,0,Casa,1,423,...,Toalha Papai Noel,natal toalha de natal toalha de mesa papai noe...,27,Lembrancinha Copa Do Mundo,160,798,1,0,0,135
4,8589941942,Caixa para 1 bis feliz natal cliente como você...,"[lembrança, personalizados, festa, caixas, cai...",2021-11-22 15:02:30,45.0,0.0,0,Lembrancinhas,30,2746,...,Caixa para 1 bis feliz natal cliente como você...,lembrança personalizados festa caixas caixinha...,1606317768786,Decoracao De Natal,344,785,4,0,0,93


In [5]:
qrels = joint_table[["qid","docno","search_position"]]
qrels = qrels.rename(columns={"search_position": "label"})
qrels

,qid,docno,label
0,178,101,281
1,176,101,281
2,21,106,86
3,135,47,160
4,93,8589941942,344
...,...,...,...
89827,223,12277,281
89828,92,17945,51
89829,142,8589938478,341
89830,69,17179881527,166


#### Atribuindo relevância

Iremos utilizar o campo "search_position" como rótulo de relevância por posição

In [6]:
# qrels["label"] = qrels["label"].apply(lambda x: int(np.round((1/x)*10000,0)))
qrels = qrels.astype({"qid": str})
qrels

,qid,docno,label
0,178,101,281
1,176,101,281
2,21,106,86
3,135,47,160
4,93,8589941942,344
...,...,...,...
89827,223,12277,281
89828,92,17945,51
89829,142,8589938478,341
89830,69,17179881527,166


# Experimentos

Utilizaremos as seguintes métricas de avaliação. De forma conjunta, nos ajudam a avaliar melhor os resultados das soluções propostas:

- Precision@5
- Precision@10
- nDCG 

Precision, até k=5 e k=10, nos dão uma ideia geral de precisão, complementada pela noção de relevância utilizada no nDCG


## Baselines

Utilizaremos como baselines modelos com tf-idf, bm25, pl2, e um pipeline conjunto de bm25 e pl2

In [7]:
index = pt.IndexFactory.of("./pd_index")
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF", verbose=True, controls={'tf_idf.k_1': 1.2, 'c': 0.1})
bm25 = pt.BatchRetrieve(index, wmodel="BM25",controls={'bm25.k_1': 1.2, 'c': 0.1, 'bm25.k_3': 0.5})
pl2 = pt.BatchRetrieve(index, wmodel="PL2")
pl2_pipeline = bm25  >> pl2


In [8]:
pt.Experiment([tf_idf, bm25, pl2,pl2_pipeline], topics, qrels, eval_metrics=["ndcg","P_10","P_5"])

BR(TF_IDF): 100%|██████████████████████████████████████████████████████| 249/249 [00:13<00:00, 17.88q/s]


,name,ndcg,P_10,P_5
0,BR(TF_IDF),0.010938,0.010843,0.012851
1,BR(BM25),0.010932,0.010442,0.014458
2,BR(PL2),0.010785,0.009639,0.013655
3,"Compose(BR(BM25), BR(PL2))",0.010820,0.009639,0.013655


## Learning to Rank

Como abordagens de learning to rank, utilizamos um pipeline utilizando RandomForest e outro utilizando XGBoost

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

train_topics, test_topics = train_test_split(topics, test_size=0.25, random_state=23)

tf = pt.BatchRetrieve(index, wmodel="TF_IDF")
pipeline = bm25 >> (tf ** pl2)
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:Tf", "WMODEL:PL2"])

rf = RandomForestRegressor(n_estimators=400)
rf_pipe = pipeline >> pt.ltr.apply_learned_model(rf)
rf_pipe.fit(train_topics, qrels)


pt.Experiment([bm25, rf_pipe], test_topics, qrels, ["ndcg","P_10","P_5"], names=["BM25 Baseline", "RandomForest"])

,name,ndcg,P_10,P_5
0,BM25 Baseline,0.010661,0.007937,0.009524
1,RandomForest,0.010709,0.007937,0.012698


In [10]:
import xgboost as xgb

bm25 = pt.BatchRetrieve(index, wmodel="BM25",controls={'bm25.k_1': 1.2, 'c': 0.1, 'bm25.k_3': 0.5})
fbr = pt.FeaturesBatchRetrieve(index, controls = {"wmodel": "BM25", 'bm25.k_1': 1.2, 'c': 0.1, 'bm25.k_3': 0.5}, features=["SAMPLE", "WMODEL:TF_IDF"]) 
lmart_x = xgb.sklearn.XGBRanker(objective='rank:ndcg',
  learning_rate=0.1,
  gamma=1.0,
  min_child_weight=0.1,
  max_depth=8,
  random_state=23)

lmart_x_pipe = fbr >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
lmart_x_pipe.fit(train_topics, qrels, test_topics, qrels)

pt.Experiment([bm25, lmart_x_pipe], test_topics, qrels, ["ndcg","P_10","P_5"], names=["BM25 Baseline", "XGBoost"])

,name,ndcg,P_10,P_5
0,BM25 Baseline,0.010661,0.007937,0.009524
1,XGBoost,0.010398,0.009524,0.009524


# Comparação de todos os modelos

In [11]:
pt.Experiment([tf_idf, bm25, pl2 ,pl2_pipeline,rf_pipe,lmart_x_pipe], test_topics, qrels, ["ndcg","P_10","P_5"], names=["TFIDF","BM25","PL2","BM25-PL2-PIPELINE","RANDOMFOREST","XGBoost"])

BR(TF_IDF): 100%|████████████████████████████████████████████████████████| 63/63 [00:03<00:00, 18.28q/s]


,name,ndcg,P_10,P_5
0,TFIDF,0.010558,0.009524,0.009524
1,BM25,0.010661,0.007937,0.009524
2,PL2,0.010508,0.004762,0.006349
3,BM25-PL2-PIPELINE,0.010457,0.004762,0.006349
4,RANDOMFOREST,0.010709,0.007937,0.012698
5,XGBoost,0.010398,0.009524,0.009524


Considerando nDCG, todos os modelos obtiveram resultados similares. Já considerando P@5 e P@10, XGBoost apresenta melhores resultados para P@10, e RandomForest para P@5. O resultado é congruente com a robustez dos modelos, mais elevada em relação aos baselines. Entretanto, os resultados das métricas são valores baixos, que podem ser melhorados utilizando modelos neurais e ensemble de modelos, conjuntamente com embeddings mais robustos para representar os documentos.